This notebook purpose is to perform static malware analysis 
Currently the scripts, defang the sample, performs strings, analysis the API used for common malicous intent and basic check if the file is packed. 
### TODO OOP
### TODO office document analysis
### TODO implement yara rules

In [5]:
import hashlib
import sys
import os
from os.path import isdir, isfile, join, basename
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from pprint import pprint
import os.path
from time import sleep
from datetime import datetime
import pefile
import requests
import bs4
import argparse
import yara

In [8]:
# Globals
Sample_dir = "Samples/"
Processed_PE_dir = "PE-samples/"
Processed_Office_dir = "Office-samples/"
now = datetime.now()
date_time = now.strftime("%m-%d-%Y-%H%M%S")

In [9]:
class Sample:
  def __init__(self, name):
          self.name = name.replace(Sample_dir, "")
          self.processed_sample_dir = ""
          self.path = ""
          self.sha256sum = ""

In [10]:
def check_dir(dir):
    if not os.path.isdir(dir):
        os.mkdir(dir)
        print("Created " + dir)
    else:
        print("Directory '" + dir + "' exists.")

In [11]:
def is_dir_empty(dir):
    dir_files = os.listdir(dir)
    if len(dir_files) == 0:
        print("Directory '" + dir + "' is empty.")
        return True

In [80]:
def Create_file(name,processed_sample_dir,data):
  if data != None:
    with open(Processed_PE_dir+processed_sample_dir+name, 'w') as newfile:
      newfile.write(data)
      newfile.close()

In [40]:
def create_processed_sample_dir(name):
    tmp_processed_sample_dir = str(date_time) + "_" + name
    new_dir = Processed_PE_dir + tmp_processed_sample_dir
    os.system("mkdir " + new_dir)
    return tmp_processed_sample_dir

In [39]:
def move_and_defang(name, Processed_sample_dir):
    suffix = ".bin"
    defanged_sample_name = name + suffix
    processed_sample = Processed_PE_dir + Processed_sample_dir + "/" + defanged_sample_name

    os.system("cp " + Sample_dir + "/" + name + " " + processed_sample)
    return processed_sample

In [87]:
def sha256sum(path, processed_sample_dir):
    sha256_hash = hashlib.sha256()
    with open(path, "rb") as f:
      hash = hashlib.sha256(f.read()).hexdigest()
      f.close()
    Create_file('/sha256sum.txt',processed_sample_dir,hash)

In [49]:
    def Get_strings(length, processed_sample_dir, path):
        file_name = "Strings.txt"
        outfile = Processed_PE_dir + processed_sample_dir + "/" + file_name
        #cmd = "flarestrings -n " + str(length) + " " + sample_path + " | rank_strings --scores > " + outfile
        cmd = "strings -n " + str(length) + " " + path + " > " + outfile
        cmd2 = "strings -el -n " + str(length) + " " + path + " >> " + outfile
        os.system(cmd)
        os.system(cmd2)

In [67]:
import requests 

API_TARGET = 'https://api.unpac.me/api/v1'
RATE_LIMIT = 5
API_KEY = 'Your api key'
auth_header={'Authorization': f'Key {API_KEY}'}

def Unpacme_upload_file(path):
  file_data = None

  files = {'file': (basename(path), file_data)}
  with open(path, "rb") as f:
			file_data = f.read()

  response = requests.post(f'{API_TARGET}/private/upload', files=files, headers=auth_header)
  response.raise_for_status()
  
  if 'id' in response.json():
    return response.json()['id']
  
  #files = {'file': (file_name, file_data)}
  #r = requests.post('https://api.unpac.me/api/v1/private/upload', files=files, headers=auth_header)
  #if not r.ok:
  #  return None
  #response = r.json()
  #return response['id'] 

def Unpacme_get_status(unpacme_id):
  response = requests.get(f'{API_TARGET}/public/status/{unpacme_id}')
  response.raise_for_status()
  return response.json().get('status', False)

def Unpacme_get_report(unpacme_id):
  response = requests.get(f'{API_TARGET}/public/results/{unpacme_id}')
  response.raise_for_status()
  return response.json()

def Unpacme_search_hash(sha256_hash):
  try:
    response = requests.get(f'{API_TARGET}/private/search/hash/{sha256_hash}', headers=auth_header)
    response.raise_for_status()
    return response.json().get('results', False)
  except requests.HTTPError as exception:
    print(exception)
  
def Unpacme_download_sample(sha256, processed_sample_dir):
  return



In [19]:
import time
import requests

api_key = 'Your api key'
base_url = 'https://analyze.intezer.com/api/v2-0'

def Inetezer_auth():
  response = requests.post(base_url + '/get-access-token', json={'api_key': api_key})
  response.raise_for_status()
  session = requests.session()
  session.headers['Authorization'] = session.headers['Authorization'] = 'Bearer %s' % response.json()['result']
  return session

def Inetezer_upload(path, session):

  with open(path, 'rb') as file_to_upload:
      files = {'file': ('file_name', file_to_upload)}
      response = session.post(base_url + '/analyze', files=files)
      assert response.status_code == 201

  while response.status_code != 200:
      time.sleep(1)
      result_url = response.json()['result_url']
      response = session.get(base_url + result_url)
      response.raise_for_status()

  print(response.json())
  return response.json()

def Intezer_search(sha256sum, session):
  response = session.get(base_url + '/files/' + sha256sum)
  print(response.json())
  return response.json()

In [22]:
def detect_packing(path,sha256sum):

  search_response = Unpacme_search_hash(sha256sum)
  #if search_response == '':
    #Do I want to upload the file to Unpacme? 
    #unpacme_id = Unpacme_upload_file(path)
    #Unpacme_get_status(unpacme_id)
    #Unpacme_get_report(unpacme_id)
    #Unpacme_download_sample(sha256,processed_sample_dir)




In [ ]:
check_dir(Sample_dir)
check_dir(Processed_PE_dir)
check_dir(Processed_Office_dir)
empty = is_dir_empty(Sample_dir)
if empty:
    raise SystemExit("Missing files in samples")

In [27]:
samples=!ls Samples/*
sample_obj = [Sample(s) for s in samples]

In [ ]:
for obj in sample_obj:
    obj.processed_sample_dir = create_processed_sample_dir(obj.name)
    obj.path = move_and_defang(obj.name, obj.processed_sample_dir)
    obj.sha256sum = sha256sum(obj.path, obj.processed_sample_dir)
    Get_strings(6, obj.processed_sample_dir, obj.path)
    Create_file("/Unpackme.txt", obj.processed_sample_dir,detect_packing(obj.path, obj.sha256sum))

Virustotal integration

In [83]:
VT_API_KEY = 'Your api key'
vt = VirusTotalPublicApi(VT_API_KEY)


if VT_API_KEY:
    for obj in sample_obj:
        print(obj.name + ":")
        print(r"SHA256sum: " + obj.sha256sum)
        parsed = json.dumps(vt.get_file_report(obj.sha256sum), indent=4)
        if(parsed != empty):
          with open(Processed_PE_dir + obj.processed_sample_dir + "/Virustotal.json", "w") as VTfile:
            VTfile.write(parsed)
            VTfile.close()
        if len(sample_obj) >= 5:
            sleep(16)

jre-8u311-windows-i586-iftw.exe:
SHA256sum: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855


In [ ]:
for obj in sample_obj:
  print(obj.name)
  print(obj.path)
  print(obj.processed_sample_dir)

In [ ]:
def malapi_check(api):
  mal_apis = {}
  pe = pefile.PE(path, fast_load=True)
  pe.parse_data_directories()
  try:
      for entry in pe.DIRECTORY_ENTRY_IMPORT:
        for imp in entry.imports:
          try:
            APICheck = requests.get("https://malapi.io/winapi/" + imp.name.decode("utf-8"))
            APISoup = bs4.BeautifulSoup(APICheck.text, 'html.parser')
            details = APISoup.select('.detail-container .content')
            ApiInfo = details[1].getText().lstrip().rstrip()
            mal_apis[imp.name.decode("utf-8")] = ApiInfo
          except:
            continue
  except:
      pass
  return mal_apis

In [ ]:
for obj in sample_obj: 
    mal_api_results =  json.dumps(check_api(obj.path), indent=2)
    with open(Processed_dir + obj.processed_sample_dir + "/Mal_api.txt", "w") as APIfile:
      APIfile.write(mal_api_results)
      APIfile.close()